In [1]:
from main import init_config
from controller import Controller
from dataset.process import TokenParser

import numpy as np

In [2]:
ctrl = Controller(*init_config())

Running the DreamProlog algorithm.
There was a problem with the provided arguments. The program will run in the default setting:
--configs prolog --logdir logdir


usage: ipykernel_launcher.py [-h] --configs CONFIGS [CONFIGS ...] --logdir
                             LOGDIR
ipykernel_launcher.py: error: the following arguments are required: --configs, --logdir
Exception ignored in: <function Controller.__del__ at 0x7f22de708c20>
Traceback (most recent call last):
  File "/home/kristsz/DreamProLog/controller.py", line 133, in __del__
    for env in self.train_envs + self.eval_envs:
  File "/home/kristsz/DreamProLog/misc/autoconfig.py", line 94, in __getattr__
    return self[name]
  File "/home/kristsz/DreamProLog/misc/autoconfig.py", line 101, in __getitem__
    if name in node.__params:
  File "/home/kristsz/DreamProLog/misc/autoconfig.py", line 91, in __getattr__
    if name == '_ConfigurationNode__params': raise AttributeError('ConfiguredModule must be initialized. Just add super().__init__()')
AttributeError: ConfiguredModule must be initialized. Just add super().__init__()


TypeError: __init__() takes 1 positional argument but 3 were given

In [ ]:
meta = ctrl.datasetManager._train_eps._meta

In [ ]:
ds = []
for v in meta.values():
    ds.extend([str(s)[1:] for s in v['action_space_text']])
    
parser = TokenParser()

def pad(narr):
    size = narr.size
    return np.pad(narr, [0, 128-size])

parsed_ds = []
for dp in ds:
    parsed_ds.append(pad(np.array(parser.parse(dp), dtype = np.int32)))

In [ ]:
for i in range(3):
    print(ds[i], parsed_ds[i], sep='\n', end='\n\n')

In [ ]:
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    while True:
        x = tf.constant([parsed_ds[np.random.randint(len(parsed_ds))] for i in range(BATCH_SIZE)])
        yield x, x
sgn = tf.TensorSpec((BATCH_SIZE, 128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))

In [ ]:
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    for data in parsed_ds:
        x = tf.constant(data)
        yield x, x
sgn = tf.TensorSpec((128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))
tf_ds = tf_ds.shuffle(300*BATCH_SIZE).batch(BATCH_SIZE)

In [ ]:
m=0
for p in parsed_ds:
    m = max(m, len(p))
print(m, len(ds))

In [ ]:
ds_it = iter(tf_ds)
next(ds_it)

In [ ]:
from transformer import EncoderLayer, DecoderLayer, MultiHeadAttention

def scaled_dot_product_attention2(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v) # (..., seq_len_q, depth_v)
    tf.print(output.shape)

    return output, attention_weights

class Encoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, querry,rate):
        super().__init__()
        self.layers = [EncoderLayer(d_model, num_heads, dff, rate) for i in range(5)]
        
        self.mha = MultiHeadAttention(d_model, num_heads)
        
        shape=(querry, d_model)
        self.variable = tf.Variable(tf.random.uniform(
            shape, minval=0, maxval=1, dtype=tf.dtypes.float32, seed=69, name=None))
        
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
                                    
    def mha(self, x):
        q, k = x, x
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        #v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        #v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        v = tf.reshape(self.variable, (1, -1, self.num_heads, self.depth))
        v = tf.transpose(v, perm=[0, 2, 1, 3])

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention2(
            q, k, v, None)
        
        print(scaled_attention.shape)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output
        
        
    def call(self, x, training):
        for layer in self.layers:
            x = layer(x,training, None)
        
        print('2', x.shape)
            
        x = self.mha(x,x, self.variable, None)
        
        return x
    
class Decoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, output_length, rate):
        super().__init__()
        self.layers = [DecoderLayer(d_model, num_heads, dff, rate) for i in range(5)]
        
        shape=(BATCH_SIZE, output_length, d_model)
        self.variable = tf.Variable(tf.random.uniform(
            shape, minval=0, maxval=1, dtype=tf.dtypes.float32, seed=69, name=None))
        
        
    def call(self, x, training):
        y = self.variable
        for layer in self.layers:
            y, _, _ = layer(y, x, training, None, None)
            
        return y
        
class Net(tf.keras.Model):
    def __init__(self, d_model, num_heads, dff):
        super().__init__()
        self.encoder = Encoder(d_model, num_heads, dff, 8, 0.04)  
        self.enc_embed = tf.keras.layers.Embedding(300, d_model)
        self.decoder = Decoder(d_model, num_heads, dff, 128, 0.04)
        
        self.dropout = tf.keras.layers.Dropout(0.08)
        
        self.dense = tf.keras.layers.Dense(300, activation='relu', use_bias=False)
        
    def call(self, x, training):
        x = self.enc_embed(x)
        x = self.encoder(x, training)
        print(x[0].shape)
        x = self.dropout(x)
        x = self.decoder(x, training)
        
        x = self.dense(x)
        
        return x

In [ ]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.1):
        super().__init__()
        self.omega = omega
        
    def call(self, y_true, y_pred):
        y = tf.nn.softmax(y_pred, axis=-1)
        y_probs = tf.gather(y, y_true, axis=-1, batch_dims=2)
        y_log_probs = tf.math.log(y_probs+0.001)
        #loss = self.omega*y_log_probs + (1-self.omega)*tf.math.cumsum(y_log_probs, axis=-1)
        loss = y_log_probs*tf.math.cumprod(0.97*tf.ones((32,128)), axis=-1)
        loss = -tf.math.reduce_sum(loss)/(128*32)
        return loss

In [ ]:
x, y_true = next(ds_it)
encoder = Encoder(256, 8, 512, 8, 0.04)
q, k = x, x
batch_size = tf.shape(q)[0]
print(q.shape)

q = encoder.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
k = encoder.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
#v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
v = tf.reshape(encoder.variable, (1, -1, self.num_heads, self.depth))
v = tf.transpose(v, perm=[0, 2, 1, 3])

In [ ]:
model = Net(256, 8, 512)
x, y_true = next(ds_it)
y_pred = model(x, False)
loss1 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss2 = Loss()
print(loss2(y_true, y_pred))
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss=Loss(), metrics=[])

In [ ]:
model.fit(tf_ds, epochs = 10)

In [ ]:
model.decoder.variable

In [ ]:

x, y_true = next(ds_it)
y_pred = model(x, False)
y_pred = tf.nn.softmax(y_pred, axis=-1)
y_probs = tf.gather(y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(y_pred, axis=-1)
print(x[:4], y_pred[:4], tf.cast(100*y_probs[:4], dtype=tf.int32), sep='\n\n')

In [ ]:
tf.Tensor(
[[10  6 13 14  3  4 73 24 14  3 73 24 13  3  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [22  6 13 39 24 14  3 38 24 14  3 46 24 14  3 37 24 14  3  4 34 24 14  3
  22  6 17 39 24 14  3 22  6 19 39 24 14  3  4 10  6 19 49 21 14 17 13  3
  48 21 14 19 17  3 48 21 14 19 13  3  4 48 21 14 16 33 19 13 17 14 13  3
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [ 4 34 24 13  3  2  3  4 96 24  7  8  3  4 22  6  9 24 13 39 24  7  8  3
  15  6 13 99 24  7  8  3 36 24 13  3 97 24 13  3 98  6 13  7  8  3  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [29 24 13  3 10  6 13 26  6 14 17  3  4 15  6 58  6 19 44 13  3 15  6 58
   6 19 45 14  3 15  6 58  6 45 44 17  3  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]], shape=(4, 128), dtype=int32)

tf.Tensor(
[[10  6 13 14  3 10 73 24  3  3 73 24 14  3  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [22  6 13 39 24 14  3 38 24 14  3 46 24 14  3 37 24 14  3  4 34 24 14  3
  22  6 17 39  6 14  3 22  6 19 39 24 14  3  4 10  6 19 49 21 14 14  6  3
  48 21 14 19 17  3 48 21 14 14 13  3  4 48 21 16 16 13  0  0 14 14 14  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [ 4  6 13  7  8  3  8  3  4  3  7  7  8 24  3  3  7  8  8  3  3  8  8 24
  24  8  3 24 24 24 24 24 24  8 24  3 24 24 24  3  3  3  7  7  3  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [10  6 13  3  6 14 13  3 29 24 17  3  4  6  6 58  6 19 44 13  4 15  6 58
   6  6 45 14  3 15  6 58 14 45  6 19 17  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]], shape=(4, 128), dtype=int64)

In [ ]:
a = tf.constant([[[2,2],[1,0]],[[1,1],[4,8]]])
b = tf.constant([[[1],[10]]])
tf.matmul(a,b)